# Offshore Wind to Hydrogen Supply Chain

### Framework For Case Studies: Single-Facility - Single-Demand Model

Importing Necessary Modules

In [1]:
from pyomo.core import *
from pyomo.environ import *
import numpy as np
import pandas as pd

import math

import time
import matplotlib.pyplot as plt

Parameters & Sets used in Model:

In [2]:
# Parameters

Times = list(np.arange(0,365))

L = 100*0.621371 #Length of Pipeline in miles from Offshore facility to Onshore

Rho = 42.4 #Density of H2 in kg/m^3 at 700 Bar
V_max = 35 #Maximum Velocity of H2 in Pipeline in m/s

deltaP_l = 0.00839*1e6 #Pressure Drop per mile of Pipeline Pa/km


# Demand

#DemandValue = 12000

#WindTurbine Costs

PCWT =  3 #Annualized Capital Cost of 15MW Wind Turbine (M$)
POWT =  1.05 #Annual Maintenance Cost of 15MW Wind Turbine (M$)

#Desalination Costs

PCDes =  47*10**(-6)*1.35 #Annualized Capital Cost of Desalination per kt of h2 produced (M$)
PODes =  11.48*10**(-6)*1.35 #Annual Maintenance Cost of Desalination per kt of h2 produced (M$)

#Electrolyzer Costs

PCEL =  45.32*10**(0) #Annualized Capital Cost of Electrolyzer per GW of Capacity (M$)
POEL =  22.250*10**(0) #Annual Maintenance Cost of Desalination per GW of Capacity (M$)

#Pipeline Costs

PCP1 =  7061*L*1e-6 #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP2 =  36*L*1e-6   #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP3 =  7694*L*1e-6 #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP4 =  34021*L*1e-6    #Annualized Capital Cost of CPipelines in M$ for L in miles

#Storage Costs

PCS =  1.56*248 #Annualized Capital Cost of Storage per kt of h2 produced (M$)
POS =  1.56*2.5/100*248 #Annual Maintenance Cost of Storage per kt of h2 produced (M$)

#Compression Costs

PCC =  1.55*82*(1e-3)*(1/1000)**0.6038 #Annualized Capital Cost of compression per GW of Compression Capacity (M$)
POC =  1.55*5/100*82*(1e-3)*(1/1000)**0.6038 #Annual Maintenance Cost of compression per GW of Compression Capacity (M$)

#Transmission Pump Costs

PCT =  1.55*82*(1e-3)*(1/1000)**0.8335 #Annualized Capital Cost of compression per GW of Transmisson Capacity (M$)
POT =  1.55*5/100*82*(1e-3)*(1/1000)**0.8335 #Annual Maintenance Cost of compression per GW of Transmisson Capacity (M$)

# Floating Platform 

FP_Cost = (500/9.8)*(0.14)  #Annualized Capital Cost of Floating Platform (M$) per GW of Electrolyzer Capacity


In [3]:
def Optimal_Design(State,CF,DemandValue):
    
    # Functions:
    
    # PWRA:
    
    def Piecewise_Reapproximate():
        A_CSA1.insert(0,model.CSA())
        A_CSA1.sort()
        B_T1.insert(A_CSA1.index(model.CSA()),math.e**(0.0787*model.CSA()**0.5))
        
        A_CSA3.insert(0,model.CSA())
        A_CSA3.sort()
        B_T3.insert(A_CSA3.index(model.CSA()),model.CSA()**0.5)

        model.del_component('model.con1')
        
        model.con1 = Piecewise(model.CPT1,model.CSA,
                        pw_pts=A_CSA1,
                        pw_constr_type='EQ',
                        f_rule=B_T1,
                        pw_repn='SOS2')
        
        model.del_component('model.con2')
        
        model.con2 = Piecewise(model.CPT3,model.CSA,
                            pw_pts=A_CSA3,
                            pw_constr_type='EQ',
                            f_rule=B_T3,
                            pw_repn='SOS2')
        
        A_Comp.insert(0,model.CCompressor())
        A_Comp.sort()
        B_Comp.insert(A_Comp.index(model.CCompressor()),model.CCompressor()**0.6038)
        
        model.del_component('model.con3')
        model.con3 = Piecewise(model.CCompressor2,model.CCompressor,
                            pw_pts=A_Comp,
                            pw_constr_type='EQ',
                            f_rule=B_Comp,
                            pw_repn='SOS2')
        
        A_Pump.insert(0,model.CPump())
        A_Pump.sort()
        B_Pump.insert(A_Pump.index(model.CPump()),model.CPump()**0.8335)

        model.del_component('model.con4')
        model.con4 = Piecewise(model.CPump2,model.CPump,
                            pw_pts=A_Pump,
                            pw_constr_type='EQ',
                            f_rule=B_Pump,
                            pw_repn='SOS2')
        
   
    
    model = ConcreteModel() 

    # Variables:

    #Decision Variables

    model.NWT = Var(initialize=1,within=Integers,bounds=(0,1000000)) #No of Wind Turbines to be installed
    model.CDes = Var(initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Desalination Capacity to be installed in H2 kt/hr
    model.CEL = Var(initialize=11,within=NonNegativeReals,bounds=(0,10**6)) #Electrolyzer Capacity to be installed in MW

    model.Cstore = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,10**6)) #LH2 Storage Capacity to be installed in kTonnes

    model.CSA = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,10**6)) #Pipeline Cross Sectional Area to be installed in inches^2


    model.CCompressor = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,10**6)) #Compressor Capacity to be installed in ?
    model.CPump = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,10**6)) #Pump Capacity to be installed in ?

    #PowerFlow Variables
    model.EWT = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power generated by Wind Turbines in each t
    model.EDes = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Desalination in each t
    model.EEL = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Electrolysis in each t
    model.ECompressor = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Compression in each t
    model.EPump = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Pumping in each t

    #Hydrogen Flow/Level Variables
    model.Lstore = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #LH2 Level in the tank in each t in ktonnes
    model.F = Var(Times,initialize=50,within=NonNegativeReals,bounds=(0,10**6)) #H2 produced in each t in ktonnes/hr
    model.M = Var(Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #H2 mass flow in pipeline each t in ktonnes/hr

    # Piece-Wise Linear Approxiamtion:
    A_CSA1 = list(np.arange(0,1100000,5000))
    B_T1 = []

    A_CSA3 = list(np.arange(0,1100000,5000))
    B_T3 = []

    for i in range(len(A_CSA1)):
        B_T1.append(math.e**(0.0787*A_CSA1[i]**0.5))

    for i in range(len(A_CSA3)):   
        B_T3.append(A_CSA3[i]**0.5)

    model.CPT1 = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,B_T1[-1]))    

    model.con1 = Piecewise(model.CPT1,model.CSA,
                        pw_pts=A_CSA1,
                        pw_constr_type='EQ',
                        f_rule=B_T1,
                        pw_repn='SOS2')

    model.CPT3 = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,B_T3[-1]))
    model.con2 = Piecewise(model.CPT3,model.CSA,
                        pw_pts=A_CSA3,
                        pw_constr_type='EQ',
                        f_rule=B_T3,
                        pw_repn='SOS2')

    A_Comp = list(np.arange(0,1100000,5000))
    B_Comp = []

    for i in range(len(A_Comp)):
        B_Comp.append(A_Comp[i]**0.6038)

    model.CCompressor2 = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,B_Comp[-1]))    

    model.con3 = Piecewise(model.CCompressor2,model.CCompressor,
                        pw_pts=A_Comp,
                        pw_constr_type='EQ',
                        f_rule=B_Comp,
                        pw_repn='SOS2')

    A_Pump = list(np.arange(0,1100000,5000))
    B_Pump = []

    for i in range(len(A_Pump)):
        B_Pump.append(A_Pump[i]**0.8335)

    model.CPump2 = Var(initialize=1,within=NonNegativeReals,bounds=(0.0001,B_Pump[-1]))

    model.con4 = Piecewise(model.CPump2,model.CPump,
                            pw_pts=A_Pump,
                            pw_constr_type='EQ',
                            f_rule=B_Pump,
                            pw_repn='SOS2')

    # Objective:

    def objrule(model):
        return (0
            +(PCWT + POWT)*model.NWT #Capex & Opex Costs of Wind Turbines
            +(PCDes + PODes)*model.CDes #Capex & Opex Costs of Desalination
            +(PCEL + POEL)*model.CEL #Capex & Opex Costs of Electrolyzer
            +(PCP1*model.CPT1 +PCP2*model.CSA +PCP3*model.CPT3 +PCP4)*1.5*1.5 #Capex & Opex Costs of Pipelines
            +(PCS +POS)*model.Cstore #Capex & Opex Costs of Storage
            +(PCC +POC)*model.CCompressor2 #Capex & Opex Costs of Compression
            +(PCT +POT)*model.CPump2 #Capex & Opex Costs of Pumping
            +FP_Cost*model.CEL #Capex Costs of Floating Platform
           )

    model.obj = Objective(rule=objrule, sense=minimize)
    
    #OOBJ:
    
    def oobj():
        val = (0
                +(PCWT + POWT)*model.NWT() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL() #Capex & Opex Costs of Electrolyzer
                +(PCP1*math.e**(0.0787*model.CSA()**(0.5)) +PCP2*model.CSA()+PCP3*model.CSA()**0.5+PCP4)*1.5*1.5 #Capex & Opex Costs of Pipelines
                +(PCS +POS)*model.Cstore() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor()**0.6038 #Capex & Opex Costs of Compression
                +(PCT +POT)*model.CPump()**0.8335 #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL() #Capex Costs of Floating Platform
                )
        return val

    def UB_Func():
        val = (0
                +(PCWT + POWT)*model.NWT() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL() #Capex & Opex Costs of Electrolyzer
                +(PCP1*model.CPT1() +PCP2*model.CSA() +PCP3*model.CPT3() +PCP4)*1.5*1.5
                +(PCS +POS)*model.Cstore() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor()**0.6038 #Capex & Opex Costs of Compression
                +(PCT +POT)*model.CPump()**0.8335 #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL() #Capex Costs of Floating Platform
                )
        return val

    def LB_Func():
        val = (0
                +(PCWT + POWT)*model.NWT() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL() #Capex & Opex Costs of Electrolyzer
                +(PCP1*math.e**(0.0787*model.CSA()**(0.5)) +PCP2*model.CSA()+PCP3*model.CSA()**0.5+PCP4)*1.5*1.5 #Capex & Opex Costs of Pipelines
                +(PCS +POS)*model.Cstore() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor2() #Capex & Opex Costs of Compression
                +(PCT +POT)*model.CPump2() #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL() #Capex Costs of Floating Platform
            )
        return val

    
    # Constraints:
    
    # Energy Balance:
    
    model.energybalance = ConstraintList()
    for t in Times:
        model.energybalance.add(model.EWT[t] == model.EDes[t] + model.EEL[t] + model.ECompressor[t] + model.EPump[t]) 

    model.windpower = ConstraintList()
    for t in Times:
        model.windpower.add(model.EWT[t] == CF[t]*0.015*24*model.NWT) 

    model.desalination = ConstraintList()
    for t in Times:
        model.desalination.add(model.EDes[t] == 52.5*10**(-3)*model.F[t]) 

    model.electrolysis = ConstraintList()
    for t in Times:
        model.electrolysis.add(model.EEL[t] == model.F[t]*50*10**(0)) 

    model.compression = ConstraintList()
    for t in Times:
        model.compression.add(model.ECompressor[t] == (model.F[t])*33.3*10**(0)/(24.5))

    model.pumping = ConstraintList()
    for t in Times:
        model.pumping.add(model.EPump[t] == 2.7e-7*model.M[t]*(deltaP_l)*(L/0.621371)/Rho)        
    
    #Capacity:

    model.c1 = ConstraintList()
    for t in Times:
        model.c1.add(model.F[t]/24 <= model.CDes) #Desalination Capacity

    model.c2 = ConstraintList()
    for t in Times:
        model.c2.add(model.EEL[t]/24 <= model.CEL) #Electrolyzer Capacity
        
    model.c3 = ConstraintList()
    for t in Times:
        model.c3.add(model.ECompressor[t]/24 <= model.CCompressor) #Compressor Capacity

    model.c4 = ConstraintList()
    for t in Times:
        model.c4.add(model.EPump[t]/24 <= model.CPump) #Pump Capacity

    # Deamamd & Inventory Constraints:

    model.mb = ConstraintList()
    for t in Times:
        if t == 0:
            model.mb.add(model.Lstore[t] == (model.Lstore[364]+model.F[t]-model.M[t]))
        else:
            model.mb.add(model.Lstore[t] == (model.Lstore[t-1]+model.F[t]-model.M[t]))
            
    model.v2 = ConstraintList()
    for t in Times:
        model.v2.add(model.Lstore[t]<=model.Cstore)

    model.v3 = ConstraintList()
    for t in Times:
        model.v3.add(model.M[t]/24 <= model.CSA*V_max*Rho*(2.34e-6)) #conversion factor from kg/s to kt/hr

    model.demand = ConstraintList()
    model.demand.add(sum(model.M[t] for t in Times) >= DemandValue)

        
    #Solution Approach: Branch & Refine:
    
    tic = time.perf_counter()
    
        #change2 start
    q1=[]
    ub1=[]
    lb1=[]
    gap1=[]
    #change2 end

    solver = SolverFactory('gurobi')
    solver.options['mipgap'] = 0.01
    solver.options['MIPFocus'] = 1
    solver.options['Heuristics'] = 1
    solver.options['cuts']=3

    solver.options['ScaleFlag']=2

    #mip_start = warmstarter()
    solver.solve(model,tee=True)

    Gap_Convex = 1-(abs(oobj())/abs(UB_Func()))
    Gap_Concave = 1-(abs(LB_Func())/abs(oobj()))

    print("Gap_Convex=",Gap_Convex)
    print("Gap_Concave=",Gap_Concave)

    while (Gap_Convex>0.00001 or Gap_Concave>0.00001):
    
        Piecewise_Reapproximate()
        
        model.del_component('model.obj')
        model.obj = Objective(rule=objrule, sense=minimize)
        
        #mip_start = warmstarter()
        #solver.solve(model,warmstart=mip_start,tee=True)
        solver.solve(model,tee=True)

        Gap_Convex = 1-(abs(oobj())/abs(UB_Func()))
        Gap_Concave = 1-(abs(LB_Func())/abs(oobj()))

        print("Gap_Convex=",Gap_Convex)
        print("Gap_Concave=",Gap_Concave)

    toc = time.perf_counter()
    print(f"Completed in {toc - tic:0.4f} seconds")
    
      
    # Export variable values to Excel
#     variable_data = []
#     for var in model.component_data_objects(Var, active=True):
#         variable_data.append([var.name, var.value])

#     df = pd.DataFrame(variable_data, columns=['Variable', 'Value'])
#     df.to_excel('variable_values.xlsx', index=False)

    model_data = {'State': State}

    # Loop through each variable in the model
    for var in model.component_data_objects(Var, active=True):
        model_data[var.name] = var.value

    model_data['Objective'] = model.obj()
    model_data['Time'] = toc - tic
    model_data['Convex_Gap'] = Gap_Convex
    model_data['Concave_Gap'] = Gap_Concave
    model_data['H2 Shipped'] = sum(model.M[t]() for t in Times)
    model_data['LCOH'] = model.obj()/sum(model.M[t]() for t in Times)
        
    model_data['cost of wind'] = (PCWT + POWT)*model.NWT()
    model_data['cost of desalination'] = (PCDes + PODes)*model.CDes()
    model_data['cost of electrolyzer'] = (PCEL + POEL)*model.CEL()
    model_data['cost of pipeline'] = (PCP1*math.e**(0.0787*model.CSA()**(0.5)) +PCP2*model.CSA()+PCP3*model.CSA()**0.5+PCP4)*1.5*1.5
    model_data['cost of storage'] = (PCS +POS)*model.Cstore()
    model_data['cost of compression'] = (PCC +POC)*model.CCompressor()**0.6038
    model_data['cost of pumping'] = (PCT +POT)*model.CPump()**0.8335
    model_data['cost of floating platform'] = FP_Cost*model.CEL()
    
    model_data['Energy Generated'] = sum(model.EWT[t]() for t in Times)
    model_data['Energy for Desalination'] = sum(model.EDes[t]() for t in Times)
    model_data['Energy for Electrolysis'] = sum(model.EEL[t]() for t in Times)
    model_data['Energy for Compression'] = sum(model.ECompressor[t]() for t in Times)
    model_data['Energy for Pumping'] = sum(model.EPump[t]() for t in Times)
    
    model_data['GHG of wind'] = (15/11)*model.NWT()*1.65e7
    model_data['GHG of desalination'] = sum(model.M[t]() for t in Times)*(1000/2*18*20)*(2.25+4.85E-1)
    model_data['GHG of electrolyzer'] = model.CEL()*1000*5.85e4
    model_data['GHG of pipeline'] = (4*model.CSA()/3.14)**0.5*2.54*10*(100/1000)*1.5*1.97E6
    model_data['GHG of storage'] = model.Cstore()*1000*1.35e5
    model_data['GHG of compression'] = model.CCompressor()*1000/32*8.71e05
    model_data['GHG of pumping'] = model.CPump()*1000/32*8.71e05
    model_data['GHG of Galvalume'] = 3.14/4*((4*model.CSA()/3.14)**0.5*2.54*10*0.001)**2*100*1000*5.55
    
    # # Append the model data to the DataFrame
    ##results_df =  results_df.append(model_data, ignore_index=True)

    # # Save the DataFrame to Excel
    ##results_df.to_excel('case_study_results.xlsx', index=False)
    
    return model_data


In [4]:
# open the excel file master_file as a pandas dataframe
master_file = pd.read_excel('master_file.xlsx', sheet_name='Sheet1')

results_df = pd.DataFrame(columns=['Model'])

# iterate through each row in the master_file dataframe
for index, row in master_file.iterrows():
    # extract the values from each row
    State =  row['Statename']
    DemandValue = row['Demand']
    buoys = row['Buoy']

    # read the file corresponding to buoys as excel from folder CF_data
    CF = pd.read_csv('CF_Data/'+str(buoys)+'.csv')

    CF = CF['CF'].tolist()

    # call the function
    print(State,buoys, DemandValue)

    model_data = Optimal_Design(State,CF,DemandValue)

   # Append the model data to the DataFrame
    results_df =  results_df.append(model_data, ignore_index=True)

# Save the DataFrame to Excel
results_df.to_excel('Fullmodel_results_Statewise_CGH2.xlsx', index=False)

final_results = results_df[['State','H2 Shipped','LCOH','Objective','cost of wind','cost of desalination','cost of electrolyzer','cost of pipeline','cost of storage','cost of compression','cost of pumping','cost of floating platform','GHG of wind','GHG of desalination','GHG of electrolyzer','GHG of pipeline','GHG of storage','GHG of compression','GHG of pumping','GHG of Galvalume','Energy Generated','Energy for Desalination','Energy for Electrolysis','Energy for Compression','Energy for Pumping']]
final_results.to_excel('Statewise_CGH2final__results.xlsx', index=False)

Alaska 46073 584.3786629400001
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file C:\Users\RISHIK~1\AppData\Local\Temp\tmp_sbasgux.pyomo.lp
Reading time = 0.05 seconds
x3812: 4759 rows, 3812 columns, 14322 nonzeros
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter Cuts to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4759 rows, 3812 columns and 14322 nonzeros
Model fingerprint: 0x1bd9c231
Model has 4 SOS constraints
Variable types: 3811 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+35]
  Objective range  [8e-05, 4e+02]
  Bounds range     [1e-04, 1e+06]
  RHS range        [1e

C:\Users\Rishi Kaashyap B\AppData\Local\Temp\ipykernel_8664\1285926706.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df =  results_df.append(model_data, ignore_index=True)


North Carolina FPSN7 1815.0101501190006
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file C:\Users\RISHIK~1\AppData\Local\Temp\tmpzgr6t4bg.pyomo.lp
Reading time = 0.03 seconds
x3812: 4759 rows, 3812 columns, 14322 nonzeros
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter Cuts to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4759 rows, 3812 columns and 14322 nonzeros
Model fingerprint: 0xe2b72e9c
Model has 4 SOS constraints
Variable types: 3811 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+35]
  Objective range  [8e-05, 4e+02]
  Bounds range     [1e-04, 1e+06]
  RHS range  

C:\Users\Rishi Kaashyap B\AppData\Local\Temp\ipykernel_8664\1285926706.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df =  results_df.append(model_data, ignore_index=True)


Maine MDRM1 324.047010205
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file C:\Users\RISHIK~1\AppData\Local\Temp\tmp1icgdsbg.pyomo.lp
Reading time = 0.07 seconds
x3812: 4759 rows, 3812 columns, 14322 nonzeros
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter Cuts to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4759 rows, 3812 columns and 14322 nonzeros
Model fingerprint: 0xbaa76a04
Model has 4 SOS constraints
Variable types: 3811 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+35]
  Objective range  [8e-05, 4e+02]
  Bounds range     [1e-04, 1e+06]
  RHS range        [1e+00, 

...

C:\Users\Rishi Kaashyap B\AppData\Local\Temp\ipykernel_8664\1285926706.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df =  results_df.append(model_data, ignore_index=True)


Indiana SGNW3 2997.5202406900007
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file C:\Users\RISHIK~1\AppData\Local\Temp\tmp178zrlgv.pyomo.lp
Reading time = 0.03 seconds
x3812: 4759 rows, 3812 columns, 14322 nonzeros
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter Cuts to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4759 rows, 3812 columns and 14322 nonzeros
Model fingerprint: 0x0995c342
Model has 4 SOS constraints
Variable types: 3811 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+35]
  Objective range  [8e-05, 4e+02]
  Bounds range     [1e-04, 1e+06]
  RHS range        [

C:\Users\Rishi Kaashyap B\AppData\Local\Temp\ipykernel_8664\1285926706.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df =  results_df.append(model_data, ignore_index=True)


Hawaii 51002 567.0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24
Read LP format model from file C:\Users\RISHIK~1\AppData\Local\Temp\tmpknkf9doj.pyomo.lp
Reading time = 0.03 seconds
x3812: 4759 rows, 3812 columns, 14322 nonzeros
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter Cuts to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4759 rows, 3812 columns and 14322 nonzeros
Model fingerprint: 0x32dfd148
Model has 4 SOS constraints
Variable types: 3811 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-03, 6e+35]
  Objective range  [8e-05, 4e+02]
  Bounds range     [1e-04, 1e+06]
  RHS range        [1e+00, 6e+02]


C:\Users\Rishi Kaashyap B\AppData\Local\Temp\ipykernel_8664\1285926706.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df =  results_df.append(model_data, ignore_index=True)
